## This program takes an input image and processes it against the stored CNN model and the co-ordinates of the parking slots and returns an image with the empty slots marked in green.

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import cv2
import os, glob
import numpy as np
## Imports for making predictions
from PIL import Image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.preprocessing import image
import pickle
#importing Config file
import import_ipynb
import Config as c

with open(c.pickle_file, 'rb') as handle:
    final_spot_dict = pickle.load(handle)

def show_images(images, cmap=None):
    cols = 2
    rows = (len(images)+1)//cols
    
    plt.figure(figsize=(15, 12))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        # use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [ ]:
# Load the Image
test_images = [plt.imread(path) for path in glob.glob(c.img_path)]
show_images(test_images)

### Use trained CNN model to make predictions

In [ ]:
cwd = os.getcwd()
top_model_weights_path = c.model_file

class_dictionary = {}
class_dictionary[0] = 'empty'
class_dictionary[1] = 'occupied'

In [ ]:
from PIL import Image
model = load_model(top_model_weights_path)

In [ ]:
def make_prediction(image):
    #Rescale image
    img = image/255.

    #Convert to a 4D tensor
    image = np.expand_dims(img, axis=0)

    # make predictions on the preloaded model
    class_predicted = model.predict(image)
    inID = np.argmax(class_predicted[0])
    label = class_dictionary[inID]
    return label

In [ ]:
empty_slots=[]
def predict_on_image(image, spot_dict = final_spot_dict, make_copy=True, color = [c.slot_color_r, c.slot_color_g, c.slot_color_b], alpha=0.5):
    if make_copy:
        new_image = np.copy(image)
        overlay = np.copy(image)
    cnt_empty = 0
    all_spots = 0
    for spot in spot_dict.keys():
        all_spots += 1
        (x1, y1, x2, y2) = spot
        (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
        #crop this image
        spot_img = image[y1:y2, x1:x2]
        spot_img = cv2.resize(spot_img, (c.test_image_width,c.test_image_height)) 
        
        label = make_prediction(spot_img)
        if label == 'empty':
            cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), color, -1)
            coordinate=x1,y1,x2,y2
            empty_slots.append(coordinate)
            cnt_empty += 1
            
    cv2.addWeighted(overlay, alpha, new_image, 1 - alpha, 0, new_image)
            
    cv2.putText(new_image, "Available: %d spots" %cnt_empty, (c.available_font_width, c.available_font_height),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (c.font_r, c.font_g, c.font_b), c.channels)
    
    cv2.putText(new_image, "Total: %d spots" %all_spots, (c.total_font_width, c.total_font_height),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (c.font_r, c.font_g, c.font_b), c.channels)
    save = False
    
    if save:
        filename = c.filename_with_marker
        cv2.imwrite(filename, new_image)
    
    return new_image


predicted_images = list(map(predict_on_image, test_images))
show_images(predicted_images)
print(empty_slots)